# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [ ]:
aircraft_type = 'cargo'  #pax or cargo

input_path = '.\\Data\\mistral_inputs.xlsx'
input_sheet = 'Echo'
output_path = '.\\Outputs\\'+ input_sheet + '_outputs'
database_path = '.\\Data\\database_mistral.xlsx'

iterations = 10000
chunks = 1000  #'auto' or more than 500

In [8]:
aircraft_type = 'cargo'  #pax or cargo

input_path = '.\\Data\\mistral_inputs.xlsx'
input_sheet = 'Bravo'
output_path = '.\\Outputs\\'+ input_sheet + '_outputs'
database_path = '.\\Data\\database_mistral.xlsx'

iterations = 10000
chunks = 1000  #'auto' or more than 500

In [7]:
from Tools import *

# Initial Definitions

Loading packages

In [ ]:
import numpy as np
import pandas as pd
import warnings
import xarray as xr
import dask.dataframe as dd
import dask.array as da
import bottleneck

## Input Parameters

Definition of Parameter and ParameterSet classes. These classes store input parameters as single and collective objects based on xarray's datatypes. Input parameters are samples randomly with a Beta-PERT distribution.

In [ ]:
class Parameter():
    """Input parameter with a Beta-PERT distribution."""
    
    def __init__(self, expected=1, minimum=1, maximum=1, name='', unit='', description=''):
        """Initialization method with values, name and unit."""
        
        self.name = name
        self.unit = unit
        self.description = description
        self.minimum = minimum
        self.expected = expected
        self.maximum = maximum
        
    def __repr__(self):
        message = f"""Parameter '{self.name}':
Minimum: {self.minimum}
Expected: {self.expected}
Maximum: {self.maximum}"""
        return message

        return 
    def PERT(self):
        """Calculates PERT values for the parameter."""
        
        if self.minimum == self.maximum:
            self.alpha = self.expected
            self.beta = self.expected
            self.mean = self.expected
        elif self.minimum > self.expected or self.maximum < self.expected:
            raise ValueError("Nominal must be equal to or between lower and upper bounds")
        else:
            self.mean = (self.minimum + 4*self.expected + self.maximum)/6
            self.alpha = 6*((self.mean - self.minimum)/(self.maximum - self.minimum))
            self.beta = 6*((self.maximum - self.mean)/(self.maximum - self.minimum))
    
    def resample(self, size=100):
        """ Samples the parameter based on the PERT values and given size."""
        
        sample = (da.random.beta(self.alpha, self.beta, size, chunks=chunks)
                  *(self.maximum - self.minimum)) + self.minimum  
        self.sample = xr.DataArray(sample, 
                            dims='i', 
                            coords=np.arange(size).reshape(1,size), 
                            name=self.name,
                            attrs={'unit': self.unit, 'description': self.description})
        return self.sample

In [ ]:
class ParameterSet():
    """A set of Parameter objetcs."""

    def __init__(self, pandas, sample_size):
        """Converts inputs DataFrame to a set of Parameter objects (a ParameterSet)."""
    
        param_dict = {}
        for variable in pandas.index:
            p = pandas.loc[variable]
            param = Parameter(p["expected"], p["minimum"],p["maximum"],
                     name=variable, unit=p["unit"], description=p["description"])
            param.PERT()
            param_dict[variable] = param
        self.parameters = param_dict
        self.sample(sample_size)
        self.to_Dataset()
    
    def __repr__(self):
        return f"{self.data}"
    
    def __getitem__(self, variable):
        return self.data[variable]
    
    def __setitem__(self, variable, value):
        self.data[variable] = value
        
    def sample(self, sample_size):
        """Samples the initial self.parameters dict of Parameters with a given number of iterations."""
        for variable in self.parameters:
            self.parameters[variable].resample(sample_size)
    
    def to_Dataset(self):
        """Converts the ParameterSet object to a xr.Dataset of samples"""
        
        dataset = xr.Dataset()
        for variable in self.parameters:
            dataset[variable] = self.parameters[variable].sample
            dataset[variable].attrs['max'] = self.parameters[variable].maximum
            dataset[variable].attrs['min'] = self.parameters[variable].minimum
            dataset[variable].attrs['exp'] = self.parameters[variable].expected
            dataset[variable].attrs['alpha'] = self.parameters[variable].alpha
            dataset[variable].attrs['beta'] = self.parameters[variable].beta
            dataset[variable].attrs['mean'] = self.parameters[variable].mean
        self.data = dataset
        
        return self.data

Definition of function to read the input file as a pd.DataFrame.

In [ ]:
def read_inputs(input_path):
    """Reads input files. Outputs DataFrame of inputs."""
    
    with pd.ExcelFile(input_path) as xlsx:
        inputs = pd.read_excel(xlsx, header = 2, index_col = 0, na_values="", usecols="B:G")
    
    inputs = inputs.rename(columns = {"nominal": "expected", 
                                            "low":"minimum", 
                                            "high":"maximum"}) #renomear título das colunas

    return inputs

Defining the function that calculates the pkm (or tkm) values that will be used throughout the code.

* `npax` is the number of passengers per flight.
* `pkm_flight` is the number of pkm of one flight.
* `pkm_year` is the number of pkm of one aircraft in one year.
* `pkm_life` is the number of pkm of the life of one aircraft.
* `pkm_fleet` is the number of pkm for all aircraft of this type that are ever produced (the fleet).

In [ ]:
def pkm(aircraft_type, p):
    """Calculates pkm (or tkm) for the aircraft."""
    if aircraft_type == "cargo":
        p["pkm_flight"] = p["payload"] * p["d_f"] * p["loadfactor"]

    elif aircraft_type == "pax":
        p["npax"] = p["seat_max"] * p["p_seat"] * p["loadfactor"]
        p["pkm_flight"] = p["npax"] * p["d_f"]

    else:
        raise Exception("Aircraft type must be 'pax' or 'cargo'")


    p["pkm_year"] = p["pkm_flight"] * p["flights_year"]
    p["pkm_life"] = p["pkm_year"] * p["lifetime"]
    p["pkm_fleet"] = p["pkm_life"] * p["fleet"]
    
    return p

## Unit Processes

For the unit processes (UP), the following function reads the excel file located on `database_path` as a pd.DataFrame.

In [ ]:
def read_unit_processes(database_path):
    """Reads unit_processes excel file from database_path as a pd.DataFrame."""
    
    with pd.ExcelFile(database_path) as xlsx:
        unit_processes = pd.read_excel(xlsx, 'UP', header=5, index_col=0, na_values=0)
        unit_processes.set_index(['compartment','subcompartment'], 
                                 append=True, inplace=True)
        
    return unit_processes

The following function transforms the pd.DataFrame to a xr.Dataset of UPs.

In [ ]:
def unit_process_dataset(pandas):
    """Transforms UPs in a pd.DataFrame to a xr.Dataset."""
    ds = xr.Dataset(pandas)
    ds = ds.rename({'dim_0':'Substances'})
    ds = ds.set_coords('unit')
    ds = ds.rename({'unit':'Units'})
    ds = ds.fillna(0)
    return ds

## LCI

Defining the LCI class, which holds the inventory of this study.

In [ ]:
class LCI():
    """Defines a LCI class based on xr.DataArray."""
    
    def __init__(self, name='', substances='', iterations=100):
        """Initialization with the phases and substances of the LCI."""
        
        self.name = name
        self.substances = substances
        self.data = None
        self.build(iterations)
  
    def __repr__(self):
        return f"{self.data}"
    
    def __getitem__(self, phase):
        return self.data[phase]
    
    def __setitem__(self, phase, other):
        self.data[phase] = other
            
    def build(self, iterations):
        """Builds the xr.DataArray for the LCI."""
        
        if self.data == None:
            self.data = xr.Dataset(coords={'Substances': self.substances, 
                                           'i': np.arange(iterations)},
                                   attrs={'Name':self.name})
            self.data.coords['Units'] = self.substances.Units
            
        return self.data
    
    def substance(self, substance):
        """Locates the specified substance on the data."""
        return self.data.loc[{'Substances': substance}]
    
    def iteration(self, iteration):
        """Locates the specified iteration on the data."""
        
        return self.data.loc[{'i': iteration}]
    
    def find(self, phase, substance, iteration):
        """Locates the specified substance, phase and iteration on the data."""
            
        return self.data[phase].loc[{'Substances': substance, 'i':iteration}]
    
    def mean(self, phase):
        """Returns the mean for all iterations of a certain phase."""
        
        return self['Office'].mean('i').load()
    
    def median(self, phase):
        """Returns the median for all iterations of a certain phase."""
        
        return self['Office'].median('i').load()

A function that can help us to build the LCI

In [ ]:
def electricity(E):
    """Calculates the LCI of electricity consumption based on a gas-wind-hydropower electricity grid."""
    
    E_wind = E * p['grid_wind']
    E_gas = E * p['grid_gas']
    E_hydro = E * p['grid_hydro']
    LCI_E = UP['Elec_wind']*E_wind + UP['Elec_gas']*E_gas + UP['Elec_hydro']*E_hydro
    
    return LCI_E

## LCIA

Some functions and classes are defined.

First, the characterization factors.

In [ ]:
def read_CF(database_path):
    """ Reads the excel file containing CFs and returns midpoint and endpoint factors."""
    
    with pd.ExcelFile(database_path) as xlsx:
        MP = pd.read_excel(xlsx, 'MP', header = 0, index_col = 0, na_values=0)
        EP = pd.read_excel(xlsx, 'EP', header = 0, index_col = 0, na_values=0)
        return MP, EP

In [ ]:
class Midpoint():
    """ Class representing midpoint characterization factors."""
    
    def __init__(self, raw):
        """ Initialize the CF class with a raw CF dataframe."""
        
        self.categories = [['freshwater eutrophication', 'FE', '$yr \cdot kg / m^3$'],
                           ['metal depletion', 'MRD', '$kg^{-1}$'],
                           ['photochemical oxidant formation', 'POF', '$kg$'], 
                           ['marine ecotoxicity', 'MET', '$m^{2} \cdot yr$'],
                           ['terrestrial acidification', 'TA','$yr \cdot m^{2}$'],
                           ['urban land occupation', 'ULO', '$m^{2} \cdot yr$'],
                           ['particulate matter formation','PMF','$kg$'],
                           ['terrestrial ecotoxicity','TET','$m^{2} \cdot yr$'],
                           ['freshwater ecotoxicity','FET','$m^{2} \cdot yr$'],
                           ['natural land transformation','NLT','$m^2$'],
                           ['ozone depletion','OD','$ppt \cdot yr$'],
                           ['marine eutrophication','ME','$yr \cdot kg / m^3$'],
                           ['agricultural land occupation','ALO','$m^{2} \cdot yr$'],
                           ['human toxicity','HT','--'],
                           ['ionising radiation','IR','$man \cdot Sv$'],
                           ['fossil depletion','FD','$MJ$'],
                           ['water depletion','WD','$m^3$'],
                           ['climate change', 'CC', '$W \cdot yr / m^{2}$']]
        self.AOP = [['Damage to Human Health', 'HH', 'yr'],
                    ['Damage to Ecosystem Diversity', 'ED', '$'],
                    ['Damage to Resource Availability', 'RA', '$']]
        self.raw = raw
        self.build()
        
    def build(self):
        self.abbrv(lst=self.categories)
        self.unit(lst=self.categories)
        self.to_dataset()
        self.to_array()
        
    def __repr__(self):
        return f"{self.array}"
    
    def __getitem__(self, item):
        return self.dataset[item]
    
    def abbrv(self, lst):
        """ Generates a category:abbreviation dictionary"""
        
        d = dict()
        for item in lst:
            d[item[0]] = item[1]
        self.abbrvs = d
        
        return self.abbrvs
    
    def unit(self, lst):
        """ Generates a abbreviation:unit dictionary."""
        
        d = dict()
        for item in lst:
            d[item[1]] = item[2]
        self.units = d
        
        return self.units
   
    def to_dataset(self):
        """Transforms the raw MP dataframe to a xr.Dataset."""

        # Adjusting the MP dataframe
        df = pd.DataFrame(index=substances.indexes["Substances"])
        for category in self.abbrvs:
            cf = self.raw.loc[category]
            cf = cf.set_index(['name','compartment','subcompartment'])
            cf = cf.squeeze()
            cf = cf.rename(category)
            df[category] = cf
        df = df.fillna(0)   

        # Creating dataset
        ds = xr.Dataset(df)

        # Renaming coords and data_vars
        ds = ds.rename({'dim_0':'Substances'})
        ds = ds.rename(self.abbrvs)
        #ds = ds.fillna(0)

        # Adding unit attributes
        for var in ds:
            ds[var].attrs['Unit'] = self.units[var]
            
        self.dataset = ds
        
        return self.dataset
    
    def to_array(self):
        """Turn the dataset to a xr.DataArray."""
        
        self.array = self.dataset.to_array()
        
        return self.array

In [ ]:
class Endpoint(Midpoint):
    """ Class representing endpoint characterization factors."""

    def build(self):
        self.unit(lst=self.AOP)
        self.to_dataset()
        self.to_array()
    
    def to_dataset(self):
        """Transforms the raw EP dataframe to a xr.Dataset."""

        # Adjusting DataFrame
        df = self.raw.rename(index=self.abbrv(lst=self.categories), columns=self.abbrv(lst=self.AOP))
        
        # Creating dataset
        ds = xr.Dataset(df)

        # Adjusting dataset
        ds = ds.rename({'dim_0':'Categories'})
        ds = ds.fillna(0)

        # Adding unit attributes
        for var in ds:
            ds[var].attrs['Unit'] = self.units[var]
            
        self.dataset = ds
        
        return self.dataset

In [ ]:
class CharactFactors():
    """Represents the Characterization Factors for both midpoint and endpoint."""
    
    def __init__(self, MP_raw, EP_raw):
        self.MP = Midpoint(MP_raw)
        self.EP = Endpoint(EP_raw)

    def __repr__(self):
        return "Characterization factors. To access them, try 'CharactFactors.MP' or '~.EP'"

Function that calculates the spearman_correlation of xr.Datasets.

In [ ]:
def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)

def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

Second, the LCIA class.

In [ ]:
class LCIA():
    """Represent LCIA results."""
    
    def __init__(self, LCI=None, CF=None, CTV=None, MP=None, EP=None):
        """Initializes the LCIA object with a LCI and CF."""
        
        self.LCI = LCI
        self.CF = CF
        self.CTV = CTV
        self.MP = MP
        self.EP = EP
        self.phases = {'Development': ['Office','Infrastructure','Capital','Prototypes','Certification'],
                       'Manufacturing': ['Materials','Factory','Logistics','Sustaining'],
                       'Operation': ['LTO','CCD','Maintenance','Airport','Fuel'],
                       'End-of-Life': ['Recycling','Landfill','Incineration']
                      }
        
    def __repr__(self):
        return f"LCIA results of the {self.MP.attrs['Name']}"
    
    def __getitem__(self, item):
        return xr.concat((self.MP[item], self.EP[item]), dim='Pathway')
    
    @classmethod
    def build(cls, LCI, CF):
        """Assesses midpoint and endpoint impacts. Returns the LCI.MP and LCI.EP datasets."""
        
        MP = (LCI.data * CF.MP.array).sum('Substances')
        MP.attrs = LCI.data.attrs
        MP = MP.rename({'variable':'Categories'})
        
        EP = (MP * CF.EP.array).sum('Categories')
        EP.attrs = MP.attrs
        EP = EP.rename({'variable':'AOP'})

        return cls(LCI=LCI, CF=CF, MP=MP, EP=EP)
    
    def SS(self, by="phase"):
        """Returns the LCA Single Score as a xr.Dataset."""
        
        cf = np.array([1.35e-2*400,9.17e-4*400,2.45e-2*200])
        ss = xr.DataArray(cf, coords={'AOP':self.EP.AOP.data}, dims='AOP')
        
        return (self.groupby(by, "EP") * ss).sum('AOP')
                         
    def mean(self, pathway="MP", by="subphase"):
        """Returns a xr.Dataset with the mean result for the chosen pathway and groupby.
        
        By can be either "sum", which returns a dataset summed over all phases;
                         "phases", which returns a dataset grouped by phases; or
                         "subphases", which returns a dataset grouped by subphases. 
        """
        
        return self.groupby(by=by, pathway=pathway).mean('i')

    def groupby(self, by="phase", pathway="MP"):
        """Returns xr.Dataset of desired pathway with aggregated life cycle phases.
        
        By can be either "sum", which returns a dataset summed over all phases;
                         "phases", which returns a dataset grouped by phases; or
                         "subphases", which returns a dataset grouped by subphases. 
        """
        
        if pathway == "MP":
            data = self.MP
        elif pathway == "EP":
            data = self.EP
        
        if by == "phase":
            ds = xr.Dataset()
            for phase in self.phases:
                ds[phase] = data[self.phases[phase]].to_array().sum('variable')
            return ds
        elif by == "sum":
            return data.to_array('Phases').sum('Phases')
        else:
            return data
            
    def build_CTV(self, parameterset):
        """Calculates the CTV of the parameterset for MP and EP."""
        
        if self.CTV == None:
            param_arr = parameterset.data.to_array('Parameters').loc[{'Parameters':[*parameterset.parameters]}]
            self.MP_array = self.MP.to_array().sum('variable').to_dataset('Categories')
            self.EP_array = self.EP.to_array().sum('variable').to_dataset('AOP')
            self.array = xr.merge([self.MP_array,self.EP_array])
            
            param_arr_nochunk = param_arr.loc[{'i':slice(999)}].chunk({'i':-1})
            array_nochunk = self.array.loc[{'i':slice(999)}].chunk({'i':-1})
            
            corr = spearman_correlation(param_arr_nochunk,array_nochunk,dim='i')
            corr_sq = corr ** 2
            corr_sum = corr_sq.sum(dim='Parameters')
            ctv = corr_sq / corr_sum
            self.CTV = ctv.fillna(0)*100
            
        return self.CTV
    
    def save(self, path, LCI=True):
        """Saves LCIA object to a NetCDF file."""
        
        
        self.MP.to_netcdf(path, group='MP', mode='w')
        self.EP.to_netcdf(path, group='EP', mode='a')
        
        if LCI:
            self.LCI.data.reset_index("Substances").to_netcdf(path, group='LCI', mode='a')
        
        if self.CTV != None:
            self.CTV.to_netcdf(path, group='CTV', mode='a')
        
        return print(f"LCIA saved at {path}")
    
    def to_excel(self, path, LCI=False):
        """Saves LCIA object to a .xlsx file."""
        
        if path[-5:] == '.xlsx':
            path = path
        else:
            path = path + '.xlsx'
        
        with pd.ExcelWriter(path) as writer:
            self.MP.to_dataframe().to_excel(writer, sheet_name='MP')
            self.EP.to_dataframe().to_excel(writer, sheet_name='EP')
            
            if self.CTV != None:
                self.CTV.to_dataframe().to_excel(writer, sheet_name='CTV')
                
            if LCI:
                self.LCI.data.reset_index("Substances").to_netcdf(path, group='LCI', mode='a')
        
        return print(f"LCIA saved at {path}")
    
    @classmethod
    def load(cls, path, chunks={}, CTV=False, LCI=False):
        """Loads NetCDF file from path.
        
        chunks: specify the chunks to pass the xr.open_dataset function.
        CTV: boolean that specifies if a CTV group if present on the NetCDF file.
        """
        
        ctv = None
        lci = None
        
        with xr.open_dataset(path, chunks=chunks, group='MP') as mp, \
             xr.open_dataset(path, chunks=chunks, group='EP') as ep:
            
            if LCI:
                with xr.open_dataset(path, chunks=chunks, group='LCI') as ds:
                    lci = ds.set_index({"Substances":["name","compartment","subcompartment"]})
        
            if CTV:
                with xr.open_dataset(path, chunks=chunks, group='CTV') as ds:
                    ctv = ds
            
            return cls(MP=mp, EP=ep, LCI=lci, CTV=ctv)
        
    
    def dist(self, pathway='MP', save=False):
        """Plots the distribution of iterations for the pathway's categories."""
        
        fig = plt.figure(figsize=(13, 19), dpi=150)
        sns.despine(left=True)
        sns.set(style="ticks", palette='GnBu', color_codes=True)

        outer = gridspec.GridSpec(6, 3, wspace=0.2, hspace=0.35)
        
        if pathway == 'MP':
            da = self.MP.to_array().sum('variable')
            lst = self.MP['Categories'].data
            name = self.MP.attrs['Name'] + "_MP_"
            finder = 'Categories'
            
        elif pathway == 'EP':
            da = self.EP.to_array().sum('variable')
            lst = self.EP['AOP'].data
            name = self.EP.attrs['Name'] + "_EP_"
            finder = 'AOP'
        
        i = 0
        for cat in lst:

            inner = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[i], 
                                                     wspace=0.1, hspace=0.1, height_ratios=(.15, .85))
            
            data = da.loc[{finder:cat}]
            
            ax = plt.Subplot(fig, inner[0])
            ax.set_xticks([])
            sns.boxplot(data, ax=ax, color='seagreen', notch=True)
            ax.set_xlabel('')
            ax.set_yticks([])
            fig.add_subplot(ax)
            i += 1

            ax = plt.Subplot(fig, inner[1])
            ax.axvline(data.mean(), 0, 1, color='darkgreen', ls= '--')
            sns.distplot(data, kde=False, color='forestgreen', ax=ax)
            ax.set_yticks([])
            ax.set_xlabel(cat, fontsize=12)
            ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0), useMathText=True)
            ax.set_xticks([data.min(), data.mean(), data.max()])
            fig.add_subplot(ax)
        
        if save:
            fig.savefig('.\\results\\' + name + 'dist.pdf', bbox_inches='tight', papertype='A3')    
        
        plt.show()
        
    
    def bar(self, pathway='MP', subphase=True, save=False):
        """Plots the percentual contribution of each life cycle phase to the pathway's categories."""
        
        if pathway == 'MP':
            lst = self.MP['Categories'].data
            name = self.MP.attrs['Name'] + "_MP_"
            finder = 'Categories'
            xlabel = "Midpoint categories"
            ylabel = "Percent of midpoint impact'"
            xticks = 18
            ymin = -20
            
        elif pathway == 'EP':
            lst = self.EP['AOP'].data
            name = self.EP.attrs['Name'] + "_EP_"
            finder = 'AOP'
            xlabel = "Endpoint Areas of Protection"
            ylabel = "Percent of endpoint impact'"
            xticks = 3
            ymin = -5
            
        
        if subphase:
            data = self.mean(pathway, by="subphase")
            name = name + "subphase_"
        else:
            data = self.mean(pathway, by="phase")
            name = name + "phase_"
        
        ds = xr.Dataset()
        for cat in lst:
            num = data.loc[{finder:cat}]
            den = abs(num.to_array()).sum()
            pct = num / den * 100
            pct = pct.to_array('Phases').drop(finder)
            ds[cat] = pct
        df = ds.to_dataframe().T
                
        sns.set(style="white", palette='deep', color_codes=False)
        ax = df.plot.bar(stacked= True, figsize=(18,8), width=0.8)
        fig = ax.get_figure();
        fig.set_dpi(150)
        ax.axhline(lw=1, color='k')
        plt.title('')
        plt.ylabel(ylabel, fontsize=13)
        plt.xlabel(xlabel, fontsize=13)
        plt.xticks(ticks=np.arange(0,xticks), rotation=0, horizontalalignment='center')
        plt.yticks(ticks=[ymin,0,20,40,60,80,100], fontsize=13)
        plt.ylim(top=100)
        plt.legend(bbox_to_anchor=(1.01, 0), loc=3, borderaxespad=0.2, edgecolor='w', fontsize=11)
        
        if save:
            fig.savefig('.\\results\\' + name + 'bar.pdf', bbox_inches='tight', papertype='A3')
            
        plt.show()
        
    def square(self, save=False):
        """Plots the CTV results for each category."""
        
        name = self.MP.attrs['Name']
        np.set_printoptions(precision=2)
        with PdfPages('.\\results\\' + name + '_CTV.pdf') as pdf:
            for cat in self.CTV:
                da = self.CTV[cat].compute()
                da = da.sortby(da, ascending=False)

                data = da.data[da.data>0]
                labels  = [f"{da[i].coords['Parameters'].data}\n{data[i].round(2)}%" for i in range(6)]
                colors = [plt.cm.Spectral(i/float(len(data))) for i in range(len(data))]

                plt.figure(figsize=(13,7), dpi=150)
                squarify.plot(data, label=labels)
                plt.title('CTV: ' + cat)
                plt.axis('off')

                if save:
                    pdf.savefig(bbox_inches='tight', papertype='A4')

                plt.show()

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)
del inputs

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

In [5]:
p.data

<xarray.Dataset>
Dimensions:             (i: 10000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 ... 9994 9995 9996 9997 9998 9999
Data variables:
    takt                (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    OEW                 (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    flights_year        (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    FH                  (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    loadfactor          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    payload             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    d_f                 (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    lifetime            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    fleet               (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    grid_wind           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    grid_gas            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    grid_hydro          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    E_office            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    water_office        (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    wastewater_office   (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    paper_use           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    devmonths           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    developers          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    car_travel          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    air_travel          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    new_factory         (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    new_machine         (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    prototypes          (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ironbirds           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    test_FH             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_Al                (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_steel             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_CFRP              (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_GFRP              (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_inconel           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_Ti                (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_Al              (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_steel           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_CFRP            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_GFRP            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_inconel         (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    b2f_Ti              (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    reuse               (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    E_factory           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    water_factory       (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    wastewater_factory  (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    lubricant           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    d_lorry             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    d_air               (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    d_sea               (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    m_lorry             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    m_air            

Then, we run the function that calculates PKM for the study.

In [8]:
p = pkm(aircraft_type, p)

<br></br>
Executing the UP reading and writing functions.

In [9]:
UP_dataframe = read_unit_processes(database_path)

In [10]:
UP = unit_process_dataset(UP_dataframe)
del UP_dataframe

<br></br>
The life cycle model uses the following phases.

In [11]:
LCI_columns =  [['DEV','DEV','DEV','DEV','DEV','MFG','MFG','MFG','MFG','OP','OP','OP','OP','OP','EOL','EOL','EOL'],\
                ['Office','Prototype','Construction','Certification','Capital','Material','Factory','Logistics','Sustaining',\
                 'LTO','CCD','Maintenance','Airport','Fuel','Recycling','Landfill','Incineration']]

phases = pd.MultiIndex.from_arrays(LCI_columns, names=['Phase', 'Subphase'])

The substances are the same as the UP dataset `UP`.

In [12]:
substances = UP.Substances

Building the empty LCI to start the calculation.

In [13]:
inventory = LCI(name=input_sheet, substances=substances, iterations=iterations)

The LCI data can be accessed by the `LCI.data` attribute.

In [14]:
inventory.data

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 10000)
Coordinates:
    Units           (Substances) object 'kg' 'm3' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconium-95'
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-urban air or from high stacks'
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
Data variables:
    *empty*
Attributes:
    Name:     Bravo

## Development

### Use of Office Buildings

Impacts of the daily work of people developing the aircraft.

Office building electricity consumption

In [15]:
def electricity(E):
    """Calculates the LCI of electricity consumption based on a gas-wind-hydropower electricity grid."""
    
    E_wind = E * p['grid_wind']
    E_gas = E * p['grid_gas']
    E_hydro = E * p['grid_hydro']
    LCI_E = UP['Elec_wind']*E_wind + UP['Elec_gas']*E_gas + UP['Elec_hydro']*E_hydro
    
    return LCI_E

In [16]:
LCI_E_office = electricity(p["E_office"])  #per month
LCI_E_office = LCI_E_office * p["devmonths"]  #per development

Office building water consumption

In [17]:
LCI_water_office = UP["Water"] * p["water_office"] \
                    + UP["Wastewater"] * p["wastewater_office"]  #per month
LCI_water_office = LCI_water_office * p["devmonths"]  #per development

Commuting and Business Travel

In [18]:
p["travel"] = 18470 / 12 * p["developers"] * p["devmonths"]  #in km

LCI_travel = UP["Car"] * p["travel"] * 0.1 + UP["Airplane"] * p["travel"] * 0.9  #per development

Use of office supplies

In [19]:
LCI_paper = UP["Paper"] * p["developers"] * p["paper_use"]  #per year
LCI_paper = LCI_paper * p["devmonths"] / 12  #per development

Total LCI for "Use of Office Building":

In [20]:
LCI_office = (LCI_E_office + LCI_water_office + LCI_paper + LCI_travel)  #per development
LCI_office = LCI_office / p["pkm_fleet"]  #per pkm

In [21]:
inventory['Office'] = LCI_office

In [22]:
del LCI_office, LCI_paper, LCI_travel, LCI_water_office, LCI_E_office

### Prototype Manufacturing

Impacts of manufacturing the prototype aircraft used during development.

*Calculated only after running the [Manufacturing and Assembly section](#mfg).*

### Certification Campaign

Impacts of the flight hours performed during development and certification.

*Calculated only after running the [Operation: Flights section](#op).*

### OEM Infrastructure Preparation

Impacts that may exists if new buildings must be built in order to manufacture the aircraft.

In [23]:
LCI_construction = (UP["Facilities"] * p["new_factory"]/2.74e5) / p["pkm_fleet"]

In [24]:
inventory["Infrastructure"] = LCI_construction
del LCI_construction

### Capital goods manufacturing

Impacts surrounding the acquisition of machines and jigs.

In [25]:
p["new_jigs"] = p["OEW"] * 500  # 50t of jigs per 100kg of product
UP["Capital"] = UP["Steel"] + UP["Jigs"]  # material plus transformation

LCI_capital = (UP["Capital"]*p["new_jigs"] + UP["Machine"]*p["new_machine"]) / p["pkm_fleet"]

In [26]:
inventory["Capital"] = LCI_capital
del LCI_capital

## Manufacturing
<a id='mfg'></a>

### Material Extraction and Transformation

Impacts surrounding the extraction and transformation of the raw materials that are used to manufacture the aircraft.

In [27]:
try:
    reuse = p['reuse']
except:
    reuse = 1

In [28]:
p["Al"] = p['p_Al'] * p['b2f_Al'] * p['OEW'] * reuse
p["steel"] = p['p_steel'] * p['b2f_steel'] * p['OEW'] * reuse
p["Ti"] = p['p_Ti'] * p['b2f_Ti'] * p['OEW'] * reuse
p["inconel"] = p['p_inconel'] * p['b2f_inconel'] * p['OEW'] * reuse
p["GFRP"] = p['p_GFRP'] * p['b2f_GFRP'] * p['OEW'] * reuse
p["CFRP"] = p['p_CFRP'] * p['b2f_CFRP'] * p['OEW'] * reuse

In [29]:
LCI_Al = UP["Aluminium"] * p["Al"]
LCI_steel = UP["Steel"] * p["steel"]
LCI_Ti = UP["Titanium"] * p["Ti"]
LCI_inconel = UP["Inconel"] * p["inconel"]
LCI_GFRP = UP["GFRP"] * p["GFRP"]
LCI_CFRP = UP["CFRP"] * p["CFRP"]

#LCI Material Extraction and Transformation
LCI_material = (LCI_Al + LCI_steel + LCI_Ti + LCI_inconel + LCI_GFRP + LCI_CFRP) / p["pkm_life"]

In [30]:
inventory["Materials"] = LCI_material
del LCI_material, LCI_Al, LCI_steel, LCI_Ti, LCI_inconel, LCI_GFRP, LCI_CFRP

### Use of Industrial Facilities

Impacts of running a factory that manufactures aircraft.

Electricity use of industrial facilities

In [31]:
LCI_E_factory = electricity(p["E_factory"])
LCI_E_factory = LCI_E_factory * p["takt"] / 30  # per aircraft

Water use of industrial facilities

In [32]:
LCI_water_factory = UP["Water"] * p["water_factory"] \
                    + UP["Wastewater"] * p["wastewater_factory"]  # per month
LCI_water_factory = LCI_water_factory * p["takt"] / 30  # per aircraft

Lubricating oils use

In [33]:
LCI_lube = UP["Lubricant"] * p["lubricant"]  # per month
LCI_lube = LCI_lube * p["takt"] / 30  # per aircraft

Industrial facilities maintenance

In [34]:
p["facilities_maint"] = p["OEW"] * 4.58e-10  # use per kg of product

LCI_facilities_maint = UP["Facilities"] * p["facilities_maint"] * 0.02  # per year
LCI_facilities_maint = LCI_facilities_maint * p["takt"] / 365  # per aircraft

Total LCI for "Use of Industrial Facilities":

In [35]:
LCI_factory = (LCI_E_factory + LCI_water_factory + LCI_lube + LCI_facilities_maint) / p["pkm_life"]

In [36]:
inventory["Factory"] = LCI_factory
del LCI_factory, LCI_E_factory, LCI_water_factory, LCI_lube, LCI_facilities_maint

### Logistics

Impacts of transporting parts and assemblies between productive sites.

In [37]:
lorry = p["d_lorry"] * p["m_lorry"] #tonne * km
sea = p["d_sea"] * p["m_sea"] #tonne * km
air = p["d_air"] * p["m_air"] #tonne * km

LCI_logistics = (UP["Lorry"]*lorry + UP["Sea"]*sea \
                + UP["Air"]*air) / p["pkm_life"]

In [38]:
inventory["Logistics"] = LCI_logistics
del LCI_logistics, lorry, sea, air

### Sustaining Engineering and Development

Impacts of maintaining an engineering workforce during serial production.

In [39]:
LCI_sustaining = inventory["Office"] * 0.01 / 30 #per day
LCI_sustaining = (LCI_sustaining * p["takt"]) / p["pkm_life"]

In [40]:
inventory["Sustaining"] = LCI_sustaining
del LCI_sustaining

## Operation
<a id='op'></a>

### Flights  

In [41]:
# CCD time

try:
    p["t_ccd"] = p["FH"]*60 - (p["t_app"] + p["t_to"] + p["t_climb"])  # minutes
except:
    p["t_ccd"] = p['FH']*60 - p['ff_lto']

p["fuel_ccd"] = p["ff_ccd"] * p["t_ccd"] * 60  # kg

In [42]:
inventory["LTO"] = UP["LTO"] / p["pkm_flight"]
inventory["CCD"] = UP["CCD"] * p["fuel_ccd"] / p["pkm_flight"]

### Aircraft Maintenance

In [43]:
LCI_maint = UP["Aluminium"]*p["maint_Al"] + UP["Steel"]*p["maint_steel"] \
            + UP["Polymer"]*p["maint_pol"] + UP["Battery"]*p['maint_battery'] #por ano

LCI_maint = (LCI_maint / p["flights_year"]) / p["pkm_flight"]

In [44]:
inventory["Maintenance"] = LCI_maint
del LCI_maint

### Airport Infrastructure

Impacts of building, operating and maintaining the airport and its surrounding infrastructure.

In [45]:
if aircraft_type == "cargo":
    ap_impact = 0.132  # 13,2% of airport impacts are due to cargo
elif aircraft_type == "pax":
    ap_impact = 0.868
else:
    ap_impact = 1

p["f_pax_ap"] = p["pax_ap"] / 22500000  # fraction of pax relative to zurich in 2000


LCI_ap = UP["Airport"] * p["f_pax_ap"]/100 / p["flights_ap"]  # 100 life years for building
LCI_ap = LCI_ap * ap_impact / p["pkm_flight"]

In [46]:
inventory["Airport"] = LCI_ap
del LCI_ap

### Fuel production

In [47]:
try:
    p["fuel_lto"] = p['ff_lto'] * p['t_lto'] * 60
except:
    p["fuel_lto"] = p['t_app']*60*p['ff_app'] + p['t_idle']*60*p['ff_idle'] \
                + p['t_to']*60*p['ff_to'] + p['t_climb']*60*p['ff_climb']
    
LCI_fuel = (UP['Kerosene'] * (p["fuel_ccd"]+p["fuel_lto"]))/ p["pkm_flight"]

In [48]:
inventory["Fuel"] = LCI_fuel
del LCI_fuel

## End-of-Life

In [49]:
try:
    reuse_factor = (2 - p['reuse'])
except:
    reuse_factor = 1

Calculating the LCI of sorting.

In [50]:
E_sort_constant = 0.4645 / 3.6  # kWh/kg of material, on average
p["E_sort"] = E_sort_constant * p['OEW'] * reuse_factor
LCI_sort = electricity(p["E_sort"])

Defining recylcing, landfilling and incineration impacts for each material.

In [51]:
materials = ['Al','steel','Ti','inconel','GFRP','CFRP']
scenarios = ['ldf', 'incin','recycl']
UP_eol = UP.rename_vars({'Landfill':'ldf','Incineration':'incin','Aluminium':'Al',
                        'Titanium':'Ti', 'Inconel':'inconel','Steel':'steel'})
eol = xr.Dataset({scenario: (['Substances','i'],da.empty((1835,iterations), chunks=(1835,chunks)))
                  for scenario in scenarios}, coords=inventory.data.coords)

In [52]:
for scenario in scenarios:
    for material in materials:
        p[scenario + "_" + material] = p["p_" + scenario + "_" + material] * p[material]  * reuse_factor
        if scenario == 'recycl':
            eol[scenario] += UP_eol[material] * p[scenario + "_" + material]
        else:
            eol[scenario] += UP_eol[scenario] * p[scenario + "_" + material]

In [53]:
inventory["Recycling"] = (LCI_sort - eol['recycl']) / p["pkm_life"]
inventory["Incineration"] = eol["incin"] / p["pkm_life"]
inventory["Landfill"] = eol["ldf"] / p["pkm_life"]

del LCI_sort, eol, materials, scenarios, UP_eol

## LCI Summary

Pending calculations from the development section.

In [54]:
MFG = inventory["Logistics"] + inventory["Sustaining"] + inventory["Factory"] + inventory["Materials"]
LCI_prot = (MFG * p["prototypes"] +  MFG * p["ironbirds"] * 0.3) / p["pkm_fleet"]

In [55]:
inventory["Prototypes"] = LCI_prot
del MFG, LCI_prot

In [56]:
p["cert_flights"] = p["test_FH"] / p["FH"]
inventory["Certification"] = (inventory["LTO"] + inventory["CCD"])*p["cert_flights"]/p["pkm_fleet"]

Final inventory for the aircraft:

In [57]:
inventory.data

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 10000)
Coordinates:
    Units           (Substances) object 'kg' 'm3' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconium-95'
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-urban air or from high stacks'
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
Data variables:
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Sustaining      (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    LTO             (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    CCD             (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Maintenance     (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Airport         (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
Attributes:
    Name:     Bravo

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [58]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

AQUI DEU PAU, VER SUBSTANCES.

In [59]:
CF = CharactFactors(MP_data, EP_data, substances)

Adjusting the NLT category CFs.

In [60]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [61]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconium-95'
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-urban air or from high stacks'
Data variables:
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 1.0 0.0456 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.56 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    PMF             (Substances) float64 0.0 0.0 0.22 0.0 ... 0.0 0.0 0.0 0.0
    TET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    NLT             (Substances) float64 -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 -0.0
    OD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ME              (Substances) float64 0.0 0.0 0.039 0.0 ... 0.0 0.0 0.0 0.0
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [62]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [63]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables:
    Office          (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Sustaining      (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    LTO             (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    CCD             (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Maintenance     (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Airport         (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Fuel            (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
Attributes:
    Name:     Bravo

In [64]:
aeco.mean("MP", by='sum')

<xarray.DataArray 'stack-eee34de82ecfd2b6c9347ad71dea4fe8' (Categories: 18)>
dask.array<mean_agg-aggregate, shape=(18,), dtype=float64, chunksize=(18,), chunktype=numpy.ndarray>
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [65]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables:
    Office          (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Sustaining      (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    LTO             (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    CCD             (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Maintenance     (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Airport         (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Fuel            (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
Attributes:
    Name:     Bravo

In [66]:
aeco.mean("EP", by="sum")

<xarray.DataArray 'stack-fd4992f297af418664185a209cc0605f' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [67]:
aeco.build_CTV(parameterset=p)
display(aeco.CTV)

KeyError: "not all values found in index 'Parameters'"

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [70]:
aeco.save(output_path, LCI=False)

LCIA saved at .\results\Bravo_outputs


In [ ]:
#aeco.to_excel(output_path)